In [1]:
from google.colab import drive
drive.mount('/content/drive') 


Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/autoencoder_features/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/autoencoder_features/tfidf_2autoencode_withstopwordchangelevels_20_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,0,0.024491,-0.016143,0.009403,-0.038202,0.045519,-0.032752,0.018270,-0.011518,-0.008610,...,0.002963,0.042562,-0.008123,-0.031473,-0.009080,0.016978,-0.005193,-0.024720,0.013811,0.041464
1,1,-0.028000,0.016008,-0.052307,-0.050315,-0.014607,0.040440,0.032949,-0.009648,-0.048885,...,0.035351,0.025425,0.028788,-0.063036,-0.020060,0.045927,0.000586,-0.047134,0.003474,-0.041898
2,2,-0.055535,0.029058,0.000450,-0.035936,0.012763,0.042501,0.023011,0.007046,0.031442,...,0.048759,-0.037616,-0.001438,-0.023432,0.059181,0.025319,-0.002322,-0.001717,0.019474,-0.049398
3,3,-0.041542,-0.007725,0.037094,-0.056550,0.039651,-0.037678,-0.004004,0.059787,0.003576,...,0.051215,-0.034003,-0.038500,0.049039,0.017030,0.012583,-0.048766,-0.027923,0.055761,-0.049178
4,4,-0.047904,0.004081,0.030327,0.021185,0.022249,0.044669,-0.046751,0.023145,-0.005052,...,0.060730,0.008289,0.043693,-0.058434,-0.018971,-0.010360,-0.052328,0.039276,0.028463,0.058594


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.024491,-0.016143,0.009403,-0.038202,0.045519,-0.032752,0.018270,-0.011518,-0.008610,0.016378,0.002963,0.042562,-0.008123,-0.031473,-0.009080,0.016978,-0.005193,-0.024720,0.013811,0.041464
1,-0.028000,0.016008,-0.052307,-0.050315,-0.014607,0.040440,0.032949,-0.009648,-0.048885,-0.001897,0.035351,0.025425,0.028788,-0.063036,-0.020060,0.045927,0.000586,-0.047134,0.003474,-0.041898
2,-0.055535,0.029058,0.000450,-0.035936,0.012763,0.042501,0.023011,0.007046,0.031442,-0.064209,0.048759,-0.037616,-0.001438,-0.023432,0.059181,0.025319,-0.002322,-0.001717,0.019474,-0.049398
3,-0.041542,-0.007725,0.037094,-0.056550,0.039651,-0.037678,-0.004004,0.059787,0.003576,0.039354,0.051215,-0.034003,-0.038500,0.049039,0.017030,0.012583,-0.048766,-0.027923,0.055761,-0.049178
4,-0.047904,0.004081,0.030327,0.021185,0.022249,0.044669,-0.046751,0.023145,-0.005052,-0.018380,0.060730,0.008289,0.043693,-0.058434,-0.018971,-0.010360,-0.052328,0.039276,0.028463,0.058594


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 20), (524, 20))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.6228646517739815


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.501639,0.501639,0.568502,0.499126,0.330153,0.330153,0.369009,0.325536


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3594    0.1465    0.2081       157
           2     0.4516    0.8270    0.5842       237
           3     0.2308    0.0462    0.0769       130

    accuracy                         0.4294       524
   macro avg     0.3473    0.3399    0.2898       524
weighted avg     0.3692    0.4294    0.3457       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.501639,0.501639,0.568502,0.499126,0.330153,0.330153,0.369009,0.325536
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.369187,0.345677


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5130    0.3823    0.4381       361
           2     0.5453    0.7722    0.6392       553
           3     0.5119    0.2810    0.3629       306

    accuracy                         0.5336      1220
   macro avg     0.5234    0.4785    0.4801      1220
weighted avg     0.5274    0.5336    0.5104      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2951    0.2293    0.2581       157
           2     0.4588    0.6582    0.5407       237
           3     0.2258    0.1077    0.1458       130

    accuracy                         0.3931       524
   macro avg     0.3266    0.3317    0.3149       524
weighted avg     0.3520    0.3931    0.3581       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-25-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.524590,0.524590,0.526185,0.491975,0.383588,0.383588,0.351703,0.351894
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.414122,0.414122,0.336930,0.338306
2,Adaboost,0.533607,0.533607,0.527387,0.510393,0.393130,0.393130,0.351954,0.358067


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9189    0.7535    0.8280       361
           2     0.7378    0.9819    0.8425       553
           3     0.9840    0.6046    0.7490       306

    accuracy                         0.8197      1220
   macro avg     0.8802    0.7800    0.8065      1220
weighted avg     0.8531    0.8197    0.8148      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3636    0.2293    0.2812       157
           2     0.4573    0.7004    0.5533       237
           3     0.2742    0.1308    0.1771       130

    accuracy                         0.4179       524
   macro avg     0.3650    0.3535    0.3372       524
weighted avg     0.3838    0.4179    0.3785       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-25-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.524590,0.524590,0.526185,0.491975,0.383588,0.383588,0.351703,0.351894
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.414122,0.414122,0.336930,0.338306
2,Adaboost,0.533607,0.533607,0.527387,0.510393,0.393130,0.393130,0.351954,0.358067
3,GBM,0.819672,0.819672,0.853143,0.814763,0.417939,0.417939,0.383810,0.378468


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6000    0.0249    0.0479       361
           2     1.0000    0.0018    0.0036       553
           3     0.2533    0.9967    0.4040       306

    accuracy                         0.2582      1220
   macro avg     0.6178    0.3412    0.1518      1220
weighted avg     0.6944    0.2582    0.1171      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2500    0.0127    0.0242       157
           2     0.0000    0.0000    0.0000       237
           3     0.2481    0.9846    0.3963       130

    accuracy                         0.2481       524
   macro avg     0.1660    0.3325    0.1402       524
weighted avg     0.1364    0.2481    0.1056       524



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.501639,0.501639,0.568502,0.499126,0.330153,0.330153,0.369009,0.325536
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.369187,0.345677
2,SVM,0.258197,0.258197,0.694358,0.117126,0.248092,0.248092,0.136447,0.105578


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4843    0.5125    0.4980       361
           2     0.5769    0.6854    0.6264       553
           3     0.4586    0.2712    0.3409       306

    accuracy                         0.5303      1220
   macro avg     0.5066    0.4897    0.4884      1220
weighted avg     0.5198    0.5303    0.5168      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2807    0.3057    0.2927       157
           2     0.4390    0.5316    0.4809       237
           3     0.2576    0.1308    0.1735       130

    accuracy                         0.3645       524
   macro avg     0.3258    0.3227    0.3157       524
weighted avg     0.3466    0.3645    0.3482       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-25-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.524590,0.524590,0.526185,0.491975,0.383588,0.383588,0.351703,0.351894
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.414122,0.414122,0.336930,0.338306
2,Adaboost,0.533607,0.533607,0.527387,0.510393,0.393130,0.393130,0.351954,0.358067
3,GBM,0.819672,0.819672,0.853143,0.814763,0.417939,0.417939,0.383810,0.378468
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.530328,0.530328,0.519800,0.516803,0.364504,0.364504,0.346572,0.348243


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4941    0.2327    0.3164       361
           2     0.4786    0.8680    0.6170       553
           3     0.2766    0.0425    0.0737       306

    accuracy                         0.4730      1220
   macro avg     0.4164    0.3811    0.3357      1220
weighted avg     0.4325    0.4730    0.3918      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2299    0.1274    0.1639       157
           2     0.4513    0.8017    0.5775       237
           3     0.1250    0.0154    0.0274       130

    accuracy                         0.4046       524
   macro avg     0.2687    0.3148    0.2563       524
weighted avg     0.3040    0.4046    0.3171       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-25-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.524590,0.524590,0.526185,0.491975,0.383588,0.383588,0.351703,0.351894
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.414122,0.414122,0.336930,0.338306
2,Adaboost,0.533607,0.533607,0.527387,0.510393,0.393130,0.393130,0.351954,0.358067
3,GBM,0.819672,0.819672,0.853143,0.814763,0.417939,0.417939,0.383810,0.378468
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.530328,0.530328,0.519800,0.516803,0.364504,0.364504,0.346572,0.348243
6,GNB,0.472951,0.472951,0.432509,0.391750,0.404580,0.404580,0.304011,0.317116
